# PRÁCTICA 2: Clasificador de noticias

### Nombres:
Introduce en esta celda los nombres de los dos integrantes del grupo:
- *Alumno 1:* DANIEL CARMONA PEDRAJAS
- *Alumno 2:* JOEL PARDO FERRERA

Objetivo: Implementar un clasificador usando el conjunto de datos recopilado de varias fuentes de internet como:

- Google News que toma noticias de varios repositorios dedicados a la información
- Periódicos:
    - El País
    - ABC
    - El Confidencial
    - 20minutos
    - El Diario

Este repositorio incluye tanto las noticias en formato '.txt' donde se almacenan los cuerpos de noticia y sus correspondientes títulos, como un '.csv' donde se contiene un registro de todas las noticias donde se refleja el número de noticias, la clase a la que pertenece (deportes, salud, ciencia y politica), el número de noticia dentro de la clase correspondiente, el título de noticia, la ruta donde está almacenada esa noticia, y por último la URL de donde se ha sacado la noticia. 

La fechas tanto de publicación como de obtención e datos se ubican en Noviembre de 2022. 

La clase a predecir es el tipo de noticia (columna 'category' de la base de datos), a partir de los archivos '.txt'. 

## 1. IMPORTACIÓN DE LIBRERÍAS

In [1]:
# git config --global user.email "jpardo0824@gmail.com"
# git config --global user.name "JPardo08"

In [2]:
# !pip3 install pandas
# !pip3 install spacy
# !pip3 install scikit-learn

In [3]:
import pandas as pd
import numpy as np
import spacy
import unicodedata
import os
#from spellchecker import SpellChecker 
#from textblob import TextBlob 
#import contractions
import re
import random
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC

## 2. CARGA DE DATOS
Cargamos los datos en dos formatos:
* DataFrame de pandas
* Generador

Para cargar los datos utilizamos la librería pandas. 
La funcion implementada recibe la ruta del archivo .csv que queramos cargar y devuelve los textos en una variable generador.

In [4]:
ruta = ".." ## CAMBIAR

In [5]:
def cargar_txts(dataframe):
    """ 
    Función para coger los documentos '.txt' de las noticias
    """
    paths = dataframe["path"].tolist()
    clases = dataframe["category"].unique().tolist()

    documentos = []

    for p in paths:
        t = p.replace(".", ruta, 1) 
        s = t.replace("/", "//")
        print(s)

        with open(s, "r", encoding="latin-1", errors='ignore') as f:
            lineas = f.readlines()
            txt1 = ''.join(lineas)
            documentos.append(txt1)
            
    return clases, documentos

In [6]:
urls_dataframe = ruta + "/Datos/urls1.csv"
path_urls_dataframe = urls_dataframe.replace("/", "//")
print(path_urls_dataframe)

# df_gen = parse(r'/Users/joelpardo/Desktop/TextClassification/Datos/urls1.csv')



noticias_dataframe = pd.read_csv(path_urls_dataframe,',')

#Visualizamos los datos
noticias_dataframe.head()

..//Datos//urls1.csv


/var/folders/yq/prfd22yn0k3gnlt89mfz3zb80000gn/T/ipykernel_1234/2303741000.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  noticias_dataframe = pd.read_csv(path_urls_dataframe,',')


,index,category,n_doc,title,path,link
0,0,sports,1,Memphis y Dumfries certifican que el 'soccer' ...,./Datos/Raw_data/sports/1.txt,https://news.google.com/__i/rss/rd/articles/CB...
1,1,sports,2,Con el dinero no basta en el fútbol - AS,./Datos/Raw_data/sports/2.txt,https://news.google.com/__i/rss/rd/articles/CB...
2,2,sports,3,El fútbol es un cuento - La Voz de Galicia,./Datos/Raw_data/sports/3.txt,https://news.google.com/__i/rss/rd/articles/CB...
3,3,sports,4,Cavani y Giménez serán sancionados pero no con...,./Datos/Raw_data/sports/4.txt,https://news.google.com/__i/rss/rd/articles/CB...
4,4,sports,5,Suspendido por una tangana a puñetazos el Alme...,./Datos/Raw_data/sports/5.txt,https://news.google.com/__i/rss/rd/articles/CB...


In [7]:
clases, documentos = cargar_txts(noticias_dataframe)
print(len(clases))
print(len(documentos))

..//Datos//Raw_data//sports//1.txt
..//Datos//Raw_data//sports//2.txt
..//Datos//Raw_data//sports//3.txt
..//Datos//Raw_data//sports//4.txt
..//Datos//Raw_data//sports//5.txt
..//Datos//Raw_data//sports//6.txt
..//Datos//Raw_data//sports//7.txt
..//Datos//Raw_data//sports//8.txt
..//Datos//Raw_data//sports//9.txt
..//Datos//Raw_data//sports//10.txt
..//Datos//Raw_data//sports//11.txt
..//Datos//Raw_data//sports//12.txt
..//Datos//Raw_data//sports//13.txt
..//Datos//Raw_data//sports//14.txt
..//Datos//Raw_data//sports//15.txt
..//Datos//Raw_data//sports//16.txt
..//Datos//Raw_data//sports//17.txt
..//Datos//Raw_data//sports//18.txt
..//Datos//Raw_data//sports//19.txt
..//Datos//Raw_data//sports//20.txt
..//Datos//Raw_data//sports//21.txt
..//Datos//Raw_data//sports//22.txt
..//Datos//Raw_data//sports//23.txt
..//Datos//Raw_data//sports//24.txt
..//Datos//Raw_data//sports//25.txt
..//Datos//Raw_data//sports//26.txt
..//Datos//Raw_data//sports//27.txt
..//Datos//Raw_data//sports//28.txt
.

Comprobamos los tipos de datos de las columnas de nuestro dataframe

In [8]:
print(type(noticias_dataframe))
# print(type(df_gen))

<class 'pandas.core.frame.DataFrame'>


In [9]:
print(documentos[0])

El lateral neerlandés regaló dos asistencias, marcó el 1-3... y el delantero azulgrana bailó con la defensa estadounidense para clasificar a la Oranje a los cuartos de final del Mundial
"En Países Bajos aprendí mucho, tomé muchos conceptos de fútbol". Gregg Berhalter, seleccionador de Estados Unidos, aseguró en la previa del partido que se inspiró en el fútbol neerlandés para fijar las bases de su ideario como entrenador. Unos conceptos futbolísticos que se dejaron ver en los primeros compases del partido, hasta que Memphis Depay decidió romper el espejismo impuesto por la caballería yankee.
Averiguaron la forma en la que tenían que atacar a Países Bajos para desarmar su entramado defensivo pero Pulisic, que no tuvo su día, no consiguió mandar al fondo de la red ni una de las dos ocasiones que tuvo en la primera parte. Una de ellas tardará mucho tiempo en olvidarla. 
Instaurados en el bloque bajo, los hombres de Louis van Gaal dieron una auténtica lección de cómo un equipo tiene que at

In [10]:
noticias_dataframe["docs"] = documentos

In [11]:
#Miramos los nombres
noticias_dataframe.columns

Index(['index', 'category', 'n_doc', 'title', 'path', 'link', 'docs'], dtype='object')

Eliminamos las filas vacías, en este caso no tenemos ninguna.

In [12]:
noticias_dataframe=noticias_dataframe.dropna()

Comprobamos que el número de instancias sigue siendo el mismo

In [13]:
print(noticias_dataframe.shape)

(240, 7)


Mostramos las posibles categorías de nuestros textos

In [14]:
clases = pd.unique(noticias_dataframe['category']).tolist()
print(clases)


['sports', 'health', 'science', 'politics']


Y comprobamos si las categorías están balanceadas en cuanto a peso.

In [15]:
frecuencias = noticias_dataframe.groupby('category').count()
frecuencias = frecuencias.sort_values('title',ascending=False)

frecuencias.head()

,index,n_doc,title,path,link,docs
category,,,,,,
health,60,60,60,60,60,60
politics,60,60,60,60,60,60
science,60,60,60,60,60,60
sports,60,60,60,60,60,60


Como podemos observar, todas las clases tienen el mismo peso, es decir, tienen el mismo numero de noticas. Tenemos un conjunto de datos homogeneo para el cual hacer la prediccion.

## 3. PRE-PROCESADO
Realizaremos el siguiente procesado:
- Separar el texto en *tokens*
- Eliminar los *tokens* de tipo *stop-word*, signos de puntuación, signos especiales o espacios
- Lematizar el texto
- Introducimos un espacio después de determinados signos de puntuación (".", "?", "%") para que el tokenizado sea correcto
- Filtramos los *tokens* con una longitud de 1

In [16]:
# !python -m spacy download es_core_news_lg

In [17]:
# https://spacy.io/models/es
nlp = spacy.load("es_core_news_lg") #Mejor modelo optimizado para la CPU

#definimos función de normalizado
def normaliza(texto):
    texto = re.sub(r"(\.)|(\?)|(\%)", r"\1 |\2 |\3 ", texto) #añadimos un espacio después de "." y "?"
    doc = nlp(texto)
    tokens = [t for t in doc if
                        len(t.text)>1 and
                       not t.is_stop and
                       not t.is_space and
                       not t.is_punct]#filtramos los tokens que nos interesan
    palabras = []
    for t in tokens:
        palabras.append(t.lemma_) #añadimos lema
    salida = ' '.join(palabras)#junta todos los tokens en un string
    
    return salida

#funcion para quitar acentos y caracteres no ASCII:

def remove_accents(input_str):     
    nfkd_form = unicodedata.normalize('NFKD', input_str)     
    only_ascii = nfkd_form.encode('ASCII', 'ignore')     
    return only_ascii.decode("utf-8", 'ignore')


Comprobamos que funciona con el elemento 0 de docs, es decir, el cuerpo de una noticia

In [18]:
#Comprobamos que funciona con el elemento 0 de noticia ("noticia")
print(noticias_dataframe['docs'][20])
print("-"*140)
remove_accents(normaliza(noticias_dataframe['docs'][20]))

Chicago está sufriendo. Pese a contar únicamente con la ausencia de Lonzo Ball (baja de larga duración), los de Illinois están siendo incapaces de encontrar su juego y anoche cayeron por 119-111 ante Golden State. Estuvo lejos de ser el peor partido de los chicos de Billy Donovan, que incluso apretaron el marcador cuando más difícil lo tenían, pero ello no evitó la que es su novena derrota en los últimos 13 encuentros.
Como decíamos, en San Francisco pelearon. Aunque los Warriors llegaron a estar hasta 18 puntos arriba al principio del último cuarto, Chicago demostró orgullo y remontó para poner un apretadísimo 110-109 a dos minutos y medio para la conclusión del choque. Hasta ahí les llegó la gasolina. Encajaron seguidamente un parcial de 7-0 que cerró definitivamente el encuentro.
En el escenario individual, el mejor de Chicago fue Nikola Vucevic, quien concluyó con un doble-doble de 23 puntos y 11 rebotes. Zach LaVine por su parte alcanzó los 21 tantos y un desconocido DeMar DeRozan

'Chicago sufrir pese contar unicamente ausencia Lonzo Ball baja largo duracion Illinois incapaz encontrar juego anoche caer 119-111 Golden State lejos partido chico Billy Donovan apretar marcador dificil tener evitar noveno derrota 13 encuentro decir San Francisco pelear Warriors llegar 18 punto principio cuarto Chicago demostrar orgullo remontar apretadisimo 110-109 minuto conclusion choque gasolina encajar seguidamente parcial 7-0 cerrar definitivamente encuentro escenario individual Chicago Nikola Vucevic concluir doble-doble 23 punto 11 rebote Zach LaVine alcanzar 21 tanto desconocido DeMar DeRozan 16 punto 15 tiro campo derrota dejar Bulls duodecimo plaza Conferencia 9-13 quedar temporada disputar el cuarto preocupacion Chicago tanto estar hablar posible traspaso dar el cambio importante plantilla turbulento inicio curso Steve Kerr acometer ajuste rotacion mostrar positivo resultado ganar compromiso Jordan Poole anoche 30 punto excelente 13 triple banquillo rotacion cambiar partid

In [19]:
#Comprobamos que funciona con el elemento 2 de noticia ("docs")
print(noticias_dataframe['docs'][2])
print("-"*140)
remove_accents(normaliza(noticias_dataframe['docs'][2]))

El fútbol es un cuento. Ya lo dijo ese Shakespeare futbolístico que es Gary Lineker, buen goleador y estrella de la tele: «El fútbol es un juego muy simple. 22 hombres persiguen un balón durante 90 minutos y al final siempre ganan los alemanes». La frase ha envejecido mal. Las mujeres también corren detrás de la pelota y arbitran (aunque sea a cuentagotas). Los alemanes ya no le ganan ni a Japón (nosotros tampoco). Y ese juego simple puede ser un tostón. Más que el fútbol, nos gusta el negocio del fútbol, esa fábrica de millonarios, algunos por lo civil y otros por lo criminal, como diría Luis Aragonés. Nos gustan su cultura, sus batallas, el cuento del fútbol.
No está tan mal un Mundial en Navidad, al margen de la tropelía de la FIFA y Catar. Con las mil y una noches persas y las luces del desierto y de las calles de Vigo. Las jóvenes iraníes jugando a tirarles el turbante a los ayatolás. Los vecinos de la calle Príncipe contando ovejas viendo pasar el carricoche de la montaña rusa po

'futbol cuento Shakespeare futbolistico Gary Lineker goleador estrella tele futbol juego simple 22 hombre perseguir balon 90 minuto ganar aleman frase envejecer mujer correr pelota arbitrar cuentagotas aleman ganar Japon juego simple toston futbol gustar negocio futbol fabrica millonario civil criminal decir Luis Aragones gustar cultura batalla cuento futbol Mundial Navidad margen tropelia FIFA Catar mil noche persa luz desierto calle Vigo joven irani jugar tirar el turbante ayatola vecino calle Principe contar oveja ver pasar carricoche montana ruso ventana Infantino Gargamel Cristiano madrastra mirar espejo Luis Enrique principito arrogante Blatter Doctor Claw malo rostro inspector Gadget acariciar gatito Messi Messi Luis Suarez llorar paisito gallego seleccionador Camerun ir casa dejar posteridad teoria peligro estar peligro realidad estar peligro estar peligro realmente estar peligro Navidad ano inaugurar Black Friday Caballero Catar-Ecuador Doha cuento'

Creamos los corpus de los textos, normalizandolos y quitando acentos

In [20]:
noticias_dataframe["corpus"] = noticias_dataframe['docs'] + noticias_dataframe['title']
corpus = noticias_dataframe["corpus"].tolist()

In [21]:
noticias_dataframe["corpus"] = noticias_dataframe["corpus"].apply(normaliza)

In [22]:
noticias_dataframe["corpus"] = noticias_dataframe["corpus"].apply(remove_accents)

Dividimos las noticias en train y test

In [23]:
noticias_dataframe.to_csv("../Datos/noticias.csv")
noticias_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     240 non-null    int64 
 1   category  240 non-null    object
 2   n_doc     240 non-null    int64 
 3   title     240 non-null    object
 4   path      240 non-null    object
 5   link      240 non-null    object
 6   docs      240 non-null    object
 7   corpus    240 non-null    object
dtypes: int64(2), object(6)
memory usage: 15.1+ KB


In [24]:
noticias_train = pd.DataFrame()
noticias_test = pd.DataFrame()

clases = noticias_dataframe["category"].unique().tolist()
p_train = 0.5

for c in clases:
    subclase = noticias_dataframe[noticias_dataframe['category'] == c]
    subclase = subclase.sample(frac = 1)

    train = subclase[:int(len(subclase)*p_train)] 

    test = subclase[int(len(subclase)*p_train):]

    noticias_train = pd.concat([noticias_train,train])
    noticias_test = pd.concat([noticias_test,test])

In [25]:
noticias_train.to_csv('../Datos/noticias_train.csv', index=False)
noticias_test.to_csv('../Datos/noticias_test.csv', index=False)

In [26]:
display(noticias_train)

,index,category,n_doc,title,path,link,docs,corpus
57,57,sports,58,El 'efecto Alonso' transforma a Aston Martin -...,./Datos/Raw_data/sports/58.txt,https://news.google.com/__i/rss/rd/articles/CB...,"Mike Krack, director del cuadro británico, con...",Mike Krack director cuadro britanico confesar ...
47,47,sports,48,"""Es el mismo Alonso que con Minardi hace 20 añ...",./Datos/Raw_data/sports/48.txt,https://news.google.com/__i/rss/rd/articles/CB...,De la Rosa defiende la valía de Fernando pese ...,Rosa defender valia Fernando pese 41 ano astur...
28,28,sports,29,Devin Booker y Jayson Tatum nombrados jugadore...,./Datos/Raw_data/sports/29.txt,https://news.google.com/__i/rss/rd/articles/CB...,La NBA acaba de publicar sus tradicionales pre...,NBA acabar publicar tradicional premio mensual...
40,40,sports,41,Ferrero: “Trabajar con Zverev no fue fácil par...,./Datos/Raw_data/sports/41.txt,https://news.google.com/__i/rss/rd/articles/CB...,Carlos Alcaraz afrontará la temporada 2023 des...,Carlos Alcaraz afrontar temporada 2023 cima te...
18,18,sports,19,Golden State acaba con Houston al ritmo de And...,./Datos/Raw_data/sports/19.txt,https://news.google.com/__i/rss/rd/articles/CB...,QuÃ© fue de aquel Andrew Wiggins que llegÃ³ a ...,QuA Andrew Wiggins llega3 objeto burla remonta...
...,...,...,...,...,...,...,...,...
224,224,politics,45,El Ministerio de Igualdad pacta conMoncloa una...,./Datos/Raw_data/politics/45.txt,https://elpais.com/espana/catalunya/2022-12-12...,"Xavier Trias, que fue alcalde de Barcelona ent...",Xavier Trias alcalde Barcelona 2011 2015 Conve...
203,203,politics,24,El Poder Judicial propone mantener laobligació...,./Datos/Raw_data/politics/24.txt,https://www.eldiario.es/politica/pp-vox-anunci...,"Las dos derechas, PP y Vox, han reaccionado de...",derecha PP Vox reaccionar forma airado viernes...
228,228,politics,49,Sandro Rosell anunciará tras Reyes si sepresen...,./Datos/Raw_data/politics/49.txt,https://elpais.com/espana/madrid/2022-12-12/al...,Javier Ortega Smith lleva un tatuaje en un bra...,Javier Ortega Smith tatuaje brazo poner COE 13...
208,208,politics,29,España y el bloque de países por un límite baj...,./Datos/Raw_data/politics/29.txt,https://www.eldiario.es/politica/fraude-fiscal...,"SegÃºn Larsen, las amenazas de âPaul Bonâ ...",segaon Larsen amenaza aPaul Bona mantener mayo...


In [27]:
display(noticias_train)

,index,category,n_doc,title,path,link,docs,corpus
57,57,sports,58,El 'efecto Alonso' transforma a Aston Martin -...,./Datos/Raw_data/sports/58.txt,https://news.google.com/__i/rss/rd/articles/CB...,"Mike Krack, director del cuadro británico, con...",Mike Krack director cuadro britanico confesar ...
47,47,sports,48,"""Es el mismo Alonso que con Minardi hace 20 añ...",./Datos/Raw_data/sports/48.txt,https://news.google.com/__i/rss/rd/articles/CB...,De la Rosa defiende la valía de Fernando pese ...,Rosa defender valia Fernando pese 41 ano astur...
28,28,sports,29,Devin Booker y Jayson Tatum nombrados jugadore...,./Datos/Raw_data/sports/29.txt,https://news.google.com/__i/rss/rd/articles/CB...,La NBA acaba de publicar sus tradicionales pre...,NBA acabar publicar tradicional premio mensual...
40,40,sports,41,Ferrero: “Trabajar con Zverev no fue fácil par...,./Datos/Raw_data/sports/41.txt,https://news.google.com/__i/rss/rd/articles/CB...,Carlos Alcaraz afrontará la temporada 2023 des...,Carlos Alcaraz afrontar temporada 2023 cima te...
18,18,sports,19,Golden State acaba con Houston al ritmo de And...,./Datos/Raw_data/sports/19.txt,https://news.google.com/__i/rss/rd/articles/CB...,QuÃ© fue de aquel Andrew Wiggins que llegÃ³ a ...,QuA Andrew Wiggins llega3 objeto burla remonta...
...,...,...,...,...,...,...,...,...
224,224,politics,45,El Ministerio de Igualdad pacta conMoncloa una...,./Datos/Raw_data/politics/45.txt,https://elpais.com/espana/catalunya/2022-12-12...,"Xavier Trias, que fue alcalde de Barcelona ent...",Xavier Trias alcalde Barcelona 2011 2015 Conve...
203,203,politics,24,El Poder Judicial propone mantener laobligació...,./Datos/Raw_data/politics/24.txt,https://www.eldiario.es/politica/pp-vox-anunci...,"Las dos derechas, PP y Vox, han reaccionado de...",derecha PP Vox reaccionar forma airado viernes...
228,228,politics,49,Sandro Rosell anunciará tras Reyes si sepresen...,./Datos/Raw_data/politics/49.txt,https://elpais.com/espana/madrid/2022-12-12/al...,Javier Ortega Smith lleva un tatuaje en un bra...,Javier Ortega Smith tatuaje brazo poner COE 13...
208,208,politics,29,España y el bloque de países por un límite baj...,./Datos/Raw_data/politics/29.txt,https://www.eldiario.es/politica/fraude-fiscal...,"SegÃºn Larsen, las amenazas de âPaul Bonâ ...",segaon Larsen amenaza aPaul Bona mantener mayo...
